<a href="https://colab.research.google.com/github/Daan0112/Bachelorthesis/blob/main/Simple_ABM_CD4_TYF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up environment

# 1. Objective  

You will build a simple day-by-day model of the CD4 response after typhoid vaccination.

From your EDA you saw that:
- TEM_response is 0 for all subjects (no CD4 TEM signal),
- the main CD4 subsets that change are TSCM, TCM, TEMRA,
- total CD4 memory (TSCM + TCM + TEMRA) rises after vaccination and stays clearly present at ~1 year.

Your model should:
- Start with many naive CD4 T cells.
-	When antigen is present, some naive cells become activated.
-	Activated cells choose between two paths:
  -	Memory path → TSCM / TCM (long-lived helper memory, slow decay),
  -	TEMRA path → TEMRA directly (terminal-like, faster decay).
-	Over time:
  -	TSCM/TCM stay relatively high even at ~1 year,
  -	TEMRA appears but decays faster.

The goal is to reproduce the shape of the median curves from your EDA, especially:

-	low signal at day 1,
-	higher TSCM/TCM/TEMRA around day 22-43,
-	clear persistence at 1 year.


# 2. Model structure

You will use 4 state variables:
- N(t) - naive CD4
- S(t) - TSCM
- C(t) - TCM
- R(t) - TEMRA

We simulate one day per step:
- t = 0, 1, 2, …, 365

Define a simple antigen function:
- A(t) = 1 for the first T_antigen days (e.g. t < 14),
- A(t) = 0 afterwards.


# 3. Model rules

Use this python template as a starting point.

## Parameters (start here)

In [ ]:
T_antigen   = 14         # duration of antigen presence (days)

p_act       = 0.0001     # fraction of N activated per day when antigen present
frac_memory = 0.7        # fraction of activated cells that follow the memory path (S/C)
                         # (1 - frac_memory) follow the TEMRA path directly (R)

# Differentiation on memory path
p_SC        = 0.05       # S -> C per day
p_CR        = 0.01       # C -> R per day  (small: most memory stays memory)

# Proliferation (while antigen present)
r_S         = 0.02       # S proliferation when antigen present
r_C         = 0.05       # C proliferation when antigen present

# Death rates (CD4-specific; memory is long-lived)
d_N         = 0.0005     # naive death
d_S         = 0.0008     # TSCM: very slow decay
d_C         = 0.0015     # TCM: slow decay
d_R         = 0.08       # TEMRA: faster decay

def antigen(t):
    """Antigen present in the early window after vaccination."""
    return 1.0 if t < T_antigen else 0.0


## Initial conditions

In [ ]:
N = 1_000_000   # mostly naive at baseline
S = 0.0         # TSCM
C = 0.0         # TCM
R = 0.0         # TEMRA

traj = {"t": [], "N": [], "S": [], "C": [], "R": []}

## Simulation loop

In [ ]:
for t in range(0, 366):  # day 0 to 365
    # Save current state
    traj["t"].append(t)
    traj["N"].append(N)
    traj["S"].append(S)
    traj["C"].append(C)
    traj["R"].append(R)

    A = antigen(t)

    # 1) activated = ALL cells leaving naive pool this day

    activated = p_act * A * N   # cells leaving N this day

    # Memory path: frac_memory of activated → will become TSCM (S)/ TCM(C)
    mem_activated = activated * frac_memory
    to_S_direct   = mem_activated * 0.6   # 60% into TSCM
    to_C_direct   = mem_activated * 0.4   # 40% directly into TCM

    # TEMRA path: rest of activated → become TEMRA (R) directly
    to_R_direct   = activated * (1 - frac_memory)

    # 2) Differentiation along the memory path
    to_C_from_S   = p_SC * S
    to_R_from_C   = p_CR * C

    # 3) Proliferation of memory cells (when antigen present)
    prolif_S      = r_S * A * S
    prolif_C      = r_C * A * C

    # 4) Death
    death_N       = d_N * N
    death_S       = d_S * S
    death_C       = d_C * C
    death_R       = d_R * R

    # 5) Updates (Euler step)
    dN = -activated - death_N
    dS = +to_S_direct + prolif_S - to_C_from_S - death_S
    dC = +to_C_direct + to_C_from_S + prolif_C - to_R_from_C - death_C
    dR = +to_R_direct + to_R_from_C - death_R

    N += dN
    S += dS
    C += dC
    R += dR


## Biological interpretation:
- Activated CD4 cells choose between:
  -	Memory path (probability = frac_memory) → TSCM/TCM (S/C).
  -	TEMRA path (probability = 1 − frac_memory) → TEMRA (R) directly.
-	A small fraction of TCM (p_CR) can later become TEMRA.
-	TSCM/TCM have slow death rates (d_S, d_C), so they persist.
-	TEMRA (R) has a faster death rate (d_R), so it contracts more.


# 4. Quick behaviour checklist

After running the simulation with the default parameters, check:

-	Day 0-5
  -	N decreases slightly.
  -	S and C start to appear (memory seeding).
  -	R begins to appear but is smaller than S+C.
-	Around day 10–25
  -	S + C + R (total CD4) reaches a peak or plateau.
-	Day 20–60
  -	R starts to decline faster.
  -	S and C remain fairly high.
-	Day 100+ / 356
  -	S and C are still clearly > 0 (persistent memory).
  -	R is lower than its early value.

If some of this is very wrong (e.g. memory disappears quickly, or grows forever), try adjusting the relevant parameters:

-	T_antigen, p_act (size/timing of response),
-	d_S, d_C (how long memory persists),
-	d_R, p_CR (how big and long TEMRA is),
-	frac_memory (how memory-dominant the response is).


# 5. Compare the model to your CD4 EDA results

Use the medians from your EDA assignment (CD4), particularly:

-	medians at:
  -	day 1
  -	~day 22
  -	~day 43
  -	~day 356/365
-	for:
  -	TSCM_response → compare with model S
  -	TCM_response → compare with model C
  -	TEMRA_response → compare with model R
  -	total_CD4 = TSCM + TCM + TEMRA → compare with model S + C + R

Steps:

-	From the model trajectories, compute:
-	total_model_CD4 = S + C + R
(You can also compute percentages if you like, but it’s not required.)
-	Make plots:
  -	For each subset (S, C, R):
        -	plot model curve vs time (0–60 days, plus the value at day 356),
        -	overlay the median data points at the same time points (1, 22, 43, 356/365).
  -	Make a separate plot for total CD4 memory:
        -	model S + C + R vs data TSCM + TCM + TEMRA medians.
-	Check:
  -	Is total CD4 memory small at day 1, higher at day 22–43, and still clearly non-zero at 1 year?
  -	Are S and C clearly present at 1 year?
  -	Is R smaller than S+C but non-zero, roughly in the same range as the TEMRA data?


# 6. Parameters to tune (by hand)

You do manual tuning, not automatic fitting.

You can try:

-	T_antigen, p_act
→ control how big and how early the response is.
-	frac_memory
→ controls how memory-dominant the subject is:
  -	high (e.g. 0.8–0.9): strong TSCM/TCM, little TEMRA,
  -	lower (e.g. 0.5–0.6): more TEMRA.
-	d_S, d_C
→ how persistent memory is at 1 year.
-	d_R, p_CR
→ how much TEMRA there is and how quickly it contracts.

Goal: find one parameter set where:

-	total memory S + C + R qualitatively matches:
  -	low at day 1,
  -	higher at day 22–43,
  -	still clearly >0 at 1 year,
-	S and C contribute most of the long-term signal,
-	R is present but not dominating.

Later, if there is time, you can vary only frac_memory to mimic different types of subjects (more memory-dominant vs more TEMRA-skewed), using your memory_fraction_peak from Task 5 of the EDA.


# 7. Deliverables

Please prepare:

-	Python notebook that:
  -	defines the model and parameters,
  -	runs the simulation from day 0 to 365,
  -	saves trajectories for N, S, C, R,
  -	produces the plots below.
-	Figures:
  -	One figure showing model trajectories for S, C, R (and optionally N) versus time.
  -	One or more figures comparing model vs data:
          -	S vs TSCM medians,
          -	C vs TCM medians,
          -	R vs TEMRA medians,
          -	total model memory (S+C+R) vs total CD4 medians.
-	Short explanation (½–1 page):
  -	List the final parameter values you used
(T_antigen, p_act, frac_memory, p_SC, p_CR, d_S, d_C, d_R).
  -	Explain:
          -	how well the model reproduces the CD4 memory persistence you saw in the data,
          -	what it gets right / wrong (e.g. TEMRA slightly too high/low),
          -	how changing frac_memory would represent more memory-dominant vs more TEMRA-skewed subjects.


# 8. What “good enough” looks like

For this first model, it is successful if:

-	The code runs from day 0 to 365 without errors.
-	You have working plots of N, S, C, R vs time.
-	The total memory curve S + C + R:
  -	is small at day 1,
  -	increases by day 22-43,
  -	and is still clearly present at ~1 year.
-	You can briefly describe in your own words:
  -	what frac_memory does,
  -	how the memory death rates (d_S, d_C) affect persistence,
  -	and how the model relates to the median curves from your EDA.

That's all we need for this step; we can refine later once this basic model is in place.
